In [1]:
from mistral.model import Transformer , ModelArgs
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
# import pytorch_lightning as pl
# from typing import Tuple
# from lightning.pytorch.callbacks import TQDMProgressBar
import math
# import torchmetrics
# from torchmetrics.regression import MeanSquaredError

In [4]:
from torchsummary import summary

In [2]:
import mmap
import random

In [3]:
from tokenizers import models, processors, trainers, Tokenizer,normalizers,pre_tokenizers,decoders
import numpy
from transformers import PreTrainedTokenizerFast, LlamaTokenizerFast
import pathlib
import os
from tokenizers import SentencePieceBPETokenizer
import sentencepiece as spm
from itertools import islice ,cycle

In [4]:
path = "../../mar-tokenizer"

In [6]:
import json

In [100]:
with open('../../mar-tokenizer/tokenizer.json', 'r') as file:
    data = json.load(file)

# Access the 'model' key and its 'vocab' array
vocab_array = data['model']['vocab']

# Count the items inside the 'vocab' array
vocab_count = len(vocab_array)

# Print the 'vocab' array and its count
print("Vocabulary Array:")
print(vocab_array)

print("\nCount of Items in Vocabulary Array:", vocab_count)

Vocabulary Array:
[['<unk>', 0.0], ['<s>', 0.0], ['</s>', 0.0], ['े', -3.198136806488037], ['ी', -3.234046697616577], ['▁', -3.6190669536590576], ['व', -3.791450023651123], ['ा', -3.8097009658813477], ['ल', -3.8552277088165283], ['र', -4.030426979064941], ['▁क', -4.096604347229004], ['.', -4.098381519317627], ['▁आहे', -4.098432540893555], ['न', -4.101080417633057], ['ो', -4.182425022125244], ['श', -4.188457012176514], ['च', -4.234373092651367], ['▁स', -4.389428615570068], [',', -4.434492588043213], ['्या', -4.54027795791626], ['त', -4.6261701583862305], ['ड', -4.655073642730713], ['ही', -4.68520975112915], ['▁केल', -4.7505269050598145], ['ू', -4.913422107696533], ['▁प', -4.955512046813965], ['ाठ', -4.957014083862305], ['म', -5.010734558105469], ['स', -5.011702537536621], ['्य', -5.055701732635498], ['ण', -5.057131290435791], ['ाल', -5.062128067016602], ['क', -5.07785177230835], ['ग', -5.083362102508545], ['▁म', -5.091159343719482], ['▁न', -5.118353843688965], ['▁अ', -5.156129837036133]

In [5]:
device = "cuda"

In [6]:
tokenizer_loaded = LlamaTokenizerFast.from_pretrained(path)

In [7]:
tokens = tokenizer_loaded("डिसेंबर ह्या दिवशी श्रीकृष्णाने अर्जुनाला गीता सांगितली.")
# tokens

In [8]:
arg = ModelArgs(dim=1024,
                n_layers=6,
                head_dim=4,
                hidden_dim=4096,
                n_heads=6,
                n_kv_heads=2,
                norm_eps=1e-6,
                max_batch_size=8,
                vocab_size=200,
                sliding_window=4,
                )

# arg = ModelArgs(dim=1024,
#                 n_layers=6,
#                 head_dim=4,
#                 hidden_dim=4096,
#                 n_heads=6,
#                 n_kv_heads=2,
#                 norm_eps=1e-6,
#                 max_batch_size=3,
#                 vocab_size=20000,
#                 sliding_window=4,
#                 )

In [35]:
dim= 1024
n_layers= 6
head_dim= 4
hidden_dim= 512
n_heads= 4
n_kv_heads= 2
norm_eps= 1e-6
vocab_size= 30000

max_batch_size= 2

# For rotary embeddings. If not set, will be infered from sliding window.
rope_theta=  None
# If this is set, use sliding window attention rotating cache.
sliding_window=  None
# If this is set, we will use MoE layers instead of dense layers.
moe=  None

In [9]:
model = Transformer(args=arg).to(device)

In [9]:
for name, param in model.named_parameters():
    print(f"{name}: {param.size()}")

tok_embeddings.weight: torch.Size([200, 1024])
norm.weight: torch.Size([1024])
output.weight: torch.Size([200, 1024])
layers.0.attention.wq.weight: torch.Size([24, 1024])
layers.0.attention.wk.weight: torch.Size([8, 1024])
layers.0.attention.wv.weight: torch.Size([8, 1024])
layers.0.attention.wo.weight: torch.Size([1024, 24])
layers.0.attention_norm.weight: torch.Size([1024])
layers.0.ffn_norm.weight: torch.Size([1024])
layers.0.feed_forward.w1.weight: torch.Size([4096, 1024])
layers.0.feed_forward.w2.weight: torch.Size([1024, 4096])
layers.0.feed_forward.w3.weight: torch.Size([4096, 1024])
layers.1.attention.wq.weight: torch.Size([24, 1024])
layers.1.attention.wk.weight: torch.Size([8, 1024])
layers.1.attention.wv.weight: torch.Size([8, 1024])
layers.1.attention.wo.weight: torch.Size([1024, 24])
layers.1.attention_norm.weight: torch.Size([1024])
layers.1.ffn_norm.weight: torch.Size([1024])
layers.1.feed_forward.w1.weight: torch.Size([4096, 1024])
layers.1.feed_forward.w2.weight: torch

In [10]:
model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
print(f"Model size: {model_size_bytes / (1024**2):.2f} MB")

Model size: 291.11 MB


In [11]:
device

'cuda'

In [53]:
torch.cuda.empty_cache()

In [93]:
import gc


In [94]:
gc.collect()

747

In [114]:
# x = torch.rand(3, 128).unsqueeze(0)
x = torch.rand(2, 2, 128).to(device)

In [116]:
x.shape

torch.Size([2, 2, 128])

In [190]:
tokens = tokenizer_loaded.encode("डिसेंबर ह्या दिवशी श्रीकृष्णाने अर्जुनाला.")

In [ ]:
tokens

In [50]:
# Convert tokens to a tensor
text_tensor = torch.tensor(tokens).to(device)

In [53]:
text_tensor.shape

torch.Size([2, 43])

In [52]:
text_tensor = torch.stack([text_tensor,text_tensor],dim=0)

In [89]:
text_tensor = torch.permute(text_tensor,(1,0) )

In [ ]:
 input_ids, seq_lengths, output_lengths = [], [], []
        for label in labels:
            prefix = f"Symptoms: {symptom}\nDisease:"
            tokens = tokenizer.encode(f"{prefix} {label}", bos=True)
            input_ids.extend(tokens)
            seq_lengths.append(len(tokens))
            output_lengths.append(len(tokens) - len(tokenizer.encode(prefix, bos=True)))
        tensor = torch.tensor(input_ids).to(model.device)
        logprobs = torch.log_softmax(model.forward(tensor, seq_lengths), dim=-1)  # sum_slens * vocab_size

In [16]:
text_tensor.shape

torch.Size([2, 43])

In [188]:
input_ids, seq_lengths, output_lengths = [], [], []

In [191]:
input_ids.extend(tokens)
seq_lengths.append(len(tokens))

In [192]:
seq_lengths

[43, 33]

In [193]:
tensor = torch.tensor(input_ids, dtype=torch.long).to(device)

In [194]:
tensor.shape

torch.Size([76])

In [145]:
(num_toks,) = text_tensor.shape

NameError: name 'text_tensor' is not defined

In [62]:
num_toks

2

In [195]:
model.eval()

logit = model.forward(input_ids=tensor, seqlens=seq_lengths)

In [121]:
loss = F.cross_entropy(logit.view(-1, logit.size(-1)), tensor.view(-1), ignore_index=-1)

In [122]:
loss

tensor(5.6331, device='cuda:0', grad_fn=<NllLossBackward0>)

In [120]:
logit.shape

torch.Size([86, 200])

In [116]:
logit.view(-1, logit.size(-1))

tensor([[ 1.0389e+00,  2.3724e-04,  3.0089e-01,  ..., -1.8550e-02,
          9.4721e-01, -1.9433e+00],
        [ 2.7117e-01, -5.8892e-01, -7.4425e-02,  ..., -5.8432e-01,
          1.7085e-02, -7.2686e-02],
        [-9.9749e-02,  7.0405e-01,  1.1654e-01,  ..., -1.3002e+00,
         -1.7605e+00, -6.0589e-01],
        ...,
        [-1.4768e-01,  4.3750e-01,  5.2466e-01,  ...,  3.8173e-01,
         -1.5919e-01, -7.7899e-01],
        [-7.5604e-01, -7.6008e-01,  6.5404e-01,  ...,  3.9160e-02,
         -5.8079e-01, -8.1597e-02],
        [ 4.1231e-01, -6.0959e-01,  5.8402e-01,  ...,  4.3537e-01,
         -2.5136e-02,  4.9881e-01]], device='cuda:0', grad_fn=<ViewBackward0>)

In [38]:
prob = torch.softmax(logit, dim=-1)

In [40]:
prob.sum()

tensor(43., device='cuda:0', grad_fn=<SumBackward0>)

In [41]:
prob

tensor([[0.0120, 0.0042, 0.0057,  ..., 0.0042, 0.0110, 0.0006],
        [0.0054, 0.0023, 0.0038,  ..., 0.0023, 0.0042, 0.0038],
        [0.0040, 0.0090, 0.0050,  ..., 0.0012, 0.0008, 0.0024],
        ...,
        [0.0033, 0.0060, 0.0065,  ..., 0.0057, 0.0033, 0.0018],
        [0.0022, 0.0022, 0.0090,  ..., 0.0049, 0.0026, 0.0043],
        [0.0065, 0.0023, 0.0077,  ..., 0.0067, 0.0042, 0.0071]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [39]:
prob.shape

torch.Size([43, 200])

In [55]:

def read_text_from_file(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
text = read_text_from_file('D:\mr_dataset\mr.txt')

In [56]:
print(len(text.split()))

174960962


In [52]:
text

['तव्यावर अर्धवट भाजून मग तळल्याने नेमके काय होते? दाल पक्वान मधले पक्वान तळतानाही हीच टीप मिळालेली आहे पण नेमके प्रयोजन कळले नाही.\n',
 '@friend१६ कृपया पाककृती लिहताना , पाककृती या लेखनप्रकाराचा वापर करावा.\n',
 '\n',
 'सगळे खूप सुंदर आहेत.\n',
 'फारच छान आहेत सगळे कुर्ते\n',
 'मस्त. ३,७,१२,१४ खूप आवडले.\n',
 '३,७,१२,१४ खूप आवडले.\n',
 'सुंदर . १ , ३ , ४ , १४ जास्त\n',
 'सुंदर . १ , ३ , ४ , १४ जास्त आवडले.\n',
 'काय सुंदर आहेत ग सगळे कुर्ते ……!\n',
 'खूप सुंदर आहेत सगळेच. ५, ७\n',
 'खूप सुंदर आहेत सगळेच. ५, ७ आणि १० तर खूपच आवडले.\n',
 'मस्तच.. ए याच्या किंमती कळतील का ग? मेल केल्यास तरी चालेल..\n',
 'एका पेक्शा एक सुंदर\n',
 'खुप सुरेख! सगळे हटके आहेत.\n',
 'सगळेच छान आहेत. हे कुठे\n',
 'सगळेच छान आहेत. हे कुठे मिळतील? आणि किमतीही कळतिल का?\n',
 '\n',
 'राफेल कराराच्या मुद्यांवर पंतप्रधान नरेंद्र मोदींनी उत्तर द्यावं :...\n',
 'मुंबई : राफेल करारावरून आता शिवसेनाही आक्रमक झाल्याचे चित्र दिेसत आहे. शिवसेनेचे...\n',
 "''मोदींनी अनिल अंबानींना 30 हजार कोटींच गिफ्ट दिलं''- राहुल गांधी\n

In [51]:
print(len(text.split()))

AttributeError: 'list' object has no attribute 'split'

In [44]:
a= text[0].split()
index = torch.randint(low=0 , high=len(a)-20, size=(8,)).tolist()

In [13]:
class TextDataset(torch.utils.data.Dataset):

    def __init__(self, file_path:str, tokenizer:LlamaTokenizerFast, context_length: int, batch_size: int):
        self.text_file = self.read_text_from_file(file_path)
        self.context_length = context_length
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.tokenized = self.text_file.split()
        self.index = torch.randint(low=0 , high=len(self.tokenized)-context_length, size=(batch_size,)).tolist()
        # self.X =[]
        # self.Y =[]
        # for idx in self.index:
        #     self.X.append(self.tokenized[idx:idx+context_length])
        #     self.Y.append(self.tokenized[idx+1:idx+1+context_length])
        # self.N = len(self.X) 
    
    
    def read_text_from_file(self,file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
        

    # def __getitem__(self, idx):
    #     return (self.X[idx], self.Y[idx])
    
    def __getitem__(self, idx):
        return (self.tokenized[idx:idx+self.context_length] , self.tokenized[idx+1:idx+1+self.context_length])

    def __len__(self):
        return 12

In [15]:
file_path = 'D:\mr_dataset\mr.txt'

In [16]:
train_data = TextDataset(file_path= file_path, tokenizer=tokenizer_loaded, context_length=256, batch_size=8 )

In [ ]:
class TextiterDataset(torch.utils.data.IterableDataset):
    def __init__(self,file_path, tokenizer:LlamaTokenizerFast, context_length: int, batch_size: int):
        self.text = self.read_text_from_file(file_path)
        self.context_length = context_length
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.tokenized = self.text_file.split()
        self.index = torch.randint(low=0 , high=len(self.tokenized)-context_length, size=(batch_size,)).tolist()
        
        
    

    def read_text_from_file(self,file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
        
    def __iter__(self):
        return (self.tokenized[idx:idx+self.context_length] , self.tokenized[idx+1:idx+1+self.context_length])

In [17]:
BATCH_SIZE = 8

train_dataloader = DataLoader(train_data , batch_size=BATCH_SIZE , shuffle=True)

# test_dataloader = DataLoader(test_data , batch_size=BATCH_SIZE , shuffle=False)

# print(f"Dataloaders: {train_dataloader, test_dataloader}")
# print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
# print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")


In [19]:
train_feature_batch, train_labels_batche = next(iter(train_dataloader))

train_feature_batch.shape ,train_labels_batche.shape

AttributeError: 'list' object has no attribute 'shape'

In [24]:
tokenizer_loaded.add_special_tokens({'pad_token': '[PAD]'})
tokenizer_loaded.padding_side = "right"
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [123]:
def get_random_chunk(filename):
    
    with open(filename, 'rb') as f:
        block_size = 30
        batch_size=8
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            #data = torch.tensor(tokenizer_loaded.encode(decoded_block), dtype=torch.long)
            # data = tokenizer_loaded.encode(decoded_block,padding='max_length' ,max_length=100)
            data = tokenizer_loaded.encode(decoded_block)
            
    return data #decoded_block

In [124]:
print(len(get_random_chunk(filename='D:\mr_dataset\mr.txt')))

68


In [125]:
print(get_random_chunk(filename='D:\mr_dataset\mr.txt'))

[1, 5, 58, 0, 16, 3, 17, 50, 87, 7, 49, 32, 34, 8, 77, 163, 36, 9, 30, 32, 8, 190, 8, 3, 18, 17, 50, 27, 3, 8, 13, 7, 140, 165, 70, 54, 3, 50, 106, 9, 120, 21, 3, 18, 17, 190, 46, 127, 7, 140, 165, 5, 136, 91, 119, 5, 167, 48, 50, 53, 187, 3, 32, 9, 5]


In [163]:
def get_batch():
    block_size = 30
    batch_size=1
    input_ids, seq_lengths, output_ids= [], [],[]
    
    
    # data = get_random_chunk(filename='D:\mr_dataset\mr.txt')
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    for _ in range(batch_size):
        data = get_random_chunk(filename='D:\mr_dataset\mr.txt')
        input_ids.extend(data[:block_size])
        output_ids.extend(data[1:block_size+1])
        seq_lengths.append(len(data))
    # input_ids.extend([data[i:i+block_size] for i in ix])
    # seq_lengths.append(len(input_ids))
    x = torch.tensor(input_ids, dtype=torch.long)
   
    # x = torch.stack([data[i:i+block_size] for i in ix])
    # for i in range(batch_size):
    #     output_ids.extend(data[i+1:i+block_size+1])
    # output_ids.extend([data[i+7:i+block_size+7] for i in ix])
    y = torch.tensor(output_ids, dtype=torch.long)
    
    # y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device), y.to(device)
    #return [data[i:i+block_size] for i in ix],[data[i+7:i+block_size+7] for i in ix]

    x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    return x,y ,seq_lengths

In [166]:
x,y,seq_lengths =get_batch()

In [167]:
x.shape, y.shape , seq_lengths

(torch.Size([30]), torch.Size([30]), [58])

In [149]:
x.dtype

torch.int64

In [36]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    X, Y = get_batch()
    logits = model(X, Y)
    
    model.train()
    return out

In [68]:
X, Y = get_batch()
X.shape, Y.shape

(torch.Size([8, 10]), torch.Size([8, 10]))

In [76]:
len(X)

8

In [133]:
logits = model(input_ids=x, seqlens=seq_lengths)

AssertionError: (529, 240)